# Gradient Stop Utilities

> For Tailwind CSS gradient utilities, daisyUI extends the from, via, and to utilities with semantic color support:

In [ ]:
#| default_exp utilities.semantic_gradients

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Union, Dict, Any
from cjm_fasthtml_tailwind.core.base import SingleValueFactory
from cjm_fasthtml_tailwind.builders.scales import SimpleFactory
from cjm_fasthtml_tailwind.utilities.backgrounds import GradientStopUtility
from cjm_fasthtml_daisyui.builders.colors import ColoredFactoryDaisyUI, SemanticColorValue, ColoredUtilityDaisyUI

from fasthtml.common import Div
from fasthtml.jupyter import JupyUvi, HTMX
from cjm_fasthtml_daisyui.core.testing import create_test_app, create_test_page, start_test_server
from cjm_fasthtml_daisyui.core.themes import DaisyUITheme
from IPython.display import display

In [ ]:
#| export
class GradientStopFactoryDaisyUI(ColoredFactoryDaisyUI):
    """Enhanced factory for gradient color stops with semantic color support."""
    
    def __init__(
        self,
        stop_type: str,  # Type of stop (from, via, to)
        doc: Optional[str] = None  # Documentation
    ):
        """Initialize gradient stop factory."""
        doc = doc or f"Gradient {stop_type} color utilities for defining the {stop_type} color of a gradient with semantic colors"
        super().__init__(stop_type, doc)
        self.stop_type = stop_type

    def __call__(
        self,
        value: Union[SemanticColorValue, str, int],  # Color, percentage, or custom value
        opacity: Optional[Union[int, str]] = None  # Optional opacity for colors
    ) -> Union[ColoredUtilityDaisyUI, GradientStopUtility]:  # Appropriate utility instance
        """Create gradient stop with color or percentage."""
        value_str = str(value)
        
        # Check if it's a percentage
        if value_str.endswith('%') or (isinstance(value, int) and 0 <= value <= 100):
            return GradientStopUtility(self.stop_type, value)
        
        # Check if it's a custom property or arbitrary value (non-color)
        if is_custom_property(value_str) or (is_arbitrary_value(value_str) and not any(c in value_str for c in ['#', 'rgb', 'hsl'])):
            return GradientStopUtility(self.stop_type, value)
        
        # Otherwise, treat as color
        return super().__call__(value, opacity)
    
    def __getattr__(
        self,
        name: str  # Attribute name
    ) -> Union[ColoredUtilityDaisyUI, GradientStopUtility]:  # Appropriate utility
        """Handle attribute access for percentages and colors."""
        # Handle percentage values (e.g., from._0, from._50, from._100)
        if name.startswith("_") and name[1:].isdigit():
            percentage = name[1:]
            return GradientStopUtility(self.stop_type, f"{percentage}%")
        
        # Otherwise, use parent's color handling
        return super().__getattr__(name)

In [ ]:
#| exports
# Create gradient stop factories with semantic color support
from_dui = GradientStopFactoryDaisyUI("from", "Gradient from color utilities for defining the starting color of a gradient with semantic colors") # Semantic gradient from color factory
via_dui = GradientStopFactoryDaisyUI("via", "Gradient via color utilities for defining the middle color of a gradient with semantic colors") # Semantic gradient via color factory
to_dui = GradientStopFactoryDaisyUI("to", "Gradient to color utilities for defining the ending color of a gradient with semantic colors") # Semantic gradient to color factory

In [ ]:
#| exports
def test_semantic_gradients_basic_examples():
    """Test gradient utilities with semantic colors."""
    # Test gradient from colors
    assert str(from_dui.primary) == "from-primary"
    assert str(from_dui.secondary) == "from-secondary"
    assert str(from_dui.base_100) == "from-base-100"
    assert str(from_dui.error) == "from-error"
    
    # Test gradient via colors
    assert str(via_dui.accent) == "via-accent"
    assert str(via_dui.neutral) == "via-neutral"
    assert str(via_dui.info) == "via-info"
    
    # Test gradient to colors
    assert str(to_dui.success) == "to-success"
    assert str(to_dui.warning) == "to-warning"
    assert str(to_dui.base_content) == "to-base-content"
    
    # Test with opacity
    assert str(from_dui.primary.opacity(50)) == "from-primary/50"
    assert str(via_dui.secondary.opacity(75)) == "via-secondary/75"
    assert str(to_dui.base_content.opacity(30)) == "to-base-content/30"

     # Test with percentages
    assert str(from_dui._0) == "from-0%"
    assert str(from_dui._50) == "from-50%"
    assert str(from_dui._100) == "from-100%"
    assert str(from_dui(25)) == "from-25%"
    assert str(from_dui("33%")) == "from-33%"
    assert str(via_dui._50) == "via-50%"
    assert str(via_dui(75)) == "via-75%"
    assert str(to_dui._100) == "to-100%"
    assert str(to_dui(90)) == "to-90%"
    
    # Test with modifiers
    assert str(from_dui.primary.hover) == "hover:from-primary"
    assert str(to_dui.error.dark) == "dark:to-error"

# Run the test
test_semantic_gradients_basic_examples()

## Practical Examples

Test with FastHTML components to show practical usage:

In [ ]:
#| exports
def test_semantic_gradients_fasthtml_examples():
    """Test practical usage patterns with FastHTML components."""
    from fasthtml.common import Div, Button, H1, P, Span
    from cjm_fasthtml_tailwind.core.base import combine_classes
    from cjm_fasthtml_tailwind.utilities.borders import border
    from cjm_fasthtml_tailwind.utilities.effects import shadow
    from cjm_fasthtml_tailwind.utilities.spacing import p
    from cjm_fasthtml_tailwind.utilities.backgrounds import bg_linear
    from cjm_fasthtml_daisyui.components.actions.button import btn
    from cjm_fasthtml_daisyui.components.data_display.card import card
    from cjm_fasthtml_daisyui.utilities.semantic_colors import text_dui
    
    # Create a gradient header with semantic colors
    gradient_header = Div(
        H1("Gradient Header", cls=str(text_dui.primary_content)),
        cls=combine_classes(
            bg_linear.to_r,
            from_dui.primary,
            to_dui.secondary,
            p(8)
        )
    )
    assert "from-primary" in gradient_header.attrs['class']
    assert "to-secondary" in gradient_header.attrs['class']

    return Div(
        gradient_header
    )

# Run the test
test_semantic_gradients_fasthtml_examples()

```html
<div>
  <div class="bg-linear-to-r from-primary to-secondary p-8">
    <h1 class="text-primary-content">Gradient Header</h1>
  </div>
</div>

```

In [ ]:
#| eval: false
#| output: false
test_func = test_semantic_gradients_fasthtml_examples
app, rt = create_test_app(theme=DaisyUITheme.LIGHT)

@rt
def index():
    return create_test_page(test_func.__doc__.title().replace('.', ''), test_func())
server = start_test_server(app)
display(HTMX())
server.stop()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()